###  convert the .mat data into the .npy data using the main key value from the main (.mat)  data files.

In [8]:
# calling the auxfunction 
import sys
import os
# Define the module path
module_path = r"E:\Projects\substructure_3d_data\Substructure_Different_DataTypes\src\modules"
if not os.path.exists(module_path):
    module_path = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\src\modules"

# Add the module path to sys.path if it's not already there
if module_path not in sys.path:
    sys.path.append(module_path)
    
import createmat2npy as mnpy
datapath = r'C:\Users\mrafik\OneDrive - C.N.R. STIIMA\tomogram all data\all_tomogram_data'
outputdatapath = r'E:\Projects\substructure_3d_data\Substructure_Different_DataTypes\data\intermdata1'
if not os.path.exists(datapath and outputdatapath):
    datapath = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\SharedContents\OneDrive - C.N.R. STIIMA\tomogram all data\all_tomogram_data"
    outputdatapath = r'C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1'

mnpy.mat2npy(datapath,outputdatapath)



 list of all files: ['aml 2 and 3', 'AML2_cell11.mat', 'AML2_cell11.npy', 'AML3_cell16.mat', 'AML3_cell16.npy', 'basicstatics.py', 'graphene data', 'metdata', 'tmpdata', 'Tomogramma_BuddingYeastCell.mat', 'Tomogramma_BuddingYeastCell.npy', 'Tomogramma_Cell1.mat', 'Tomogramma_Cell1.npy', 'Tomogramma_Cell2.mat', 'Tomogramma_Cell2.npy', 'Tomogramma_Cell3.mat', 'Tomogramma_Cell3.npy', 'tomo_Grafene_24h.mat', 'tomo_Grafene_24h.npy', 'tomo_grafene_48h.mat', 'tomo_grafene_48h.npy', 'yeast cell in various stages', 'yeast_data.csv', '__pycache__']
output directory already exist 

AML2_cell11.mat
Shape of Data: (200, 200, 200)
Data Type: float64
Min Value: 1.3312461759020122
Max Value: 1.3847334887289884
Mean Value: 1.3346517690936042
Standard Deviation: 0.00363229717980464
AML3_cell16.mat
Shape of Data: (200, 200, 200)
Data Type: float64
Min Value: 1.3311161422135425
Max Value: 1.3968166112149256
Mean Value: 1.3347910557041864
Standard Deviation: 0.004951461534242804
Tomogramma_BuddingYeastCell

In [9]:
# pip install seaborn
# pip install scikit-learn

In [32]:
#  Python Script for Loading & Normalization
    
# Import necessary libraries
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import json
output_folder = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\processed"
# Define a function to load and normalize 3D Numpy data
def load_and_normalize_npy(file_path):
    """
    Loads a .npy file and applies Min-Max normalization.
    """
    data = np.load(file_path)  # Load the file
    min_val, max_val = np.min(data), np.max(data)  # Get min-max values
    normalized_data = (data - min_val) / (max_val - min_val)  # Normalize to [0,1]
    return normalized_data, min_val, max_val

# Set the directory containing .npy files (update this with your folder path)
# data_folder = "your_directory_path_here"
data_folder = outputdatapath

npy_files = glob.glob(os.path.join(data_folder, "*.npy"))  # List all .npy files

# Dictionary to store the normalized datasets
normalized_data = {}
data_ranges = {}

# Load and normalize each dataset
for file in npy_files:
    file_name = os.path.basename(file)
    data, min_val, max_val = load_and_normalize_npy(file)
    normalized_data[file_name] = data  # Store in dictionary
    data_ranges[file_name] = (min_val, max_val)  # Store original data range
    print(f"Loaded and normalized {file_name} - Min: {min_val}, Max: {max_val}")


data_dict_np = normalized_data
import scipy.io as sio
# Save dictionary as `.mat`
mat_path = os.path.join(output_folder, "all_normalizeddata.mat")
sio.savemat(mat_path, data_dict_np)
print(f"✅ Dictionary saved as MATLAB `.mat`: {mat_path}")

# Load dictionary from `.mat`
loaded_mat = sio.loadmat(mat_path)
# print(f"📂 Loaded dictionary: {loaded_mat}")

# Plot histogram of one dataset (Example: First File)
# sample_file = list(normalized_data.keys())[0]
# sample_data = normalized_data[sample_file].flatten()
# plt.figure(figsize=(6, 4))
# sns.histplot(sample_data, bins=50, kde=True)
# plt.title(f"Distribution of Normalized Data: {sample_file}")
# plt.xlabel("Normalized Value")
# plt.ylabel("Frequency")
# plt.show()


Loaded and normalized AML2_cell11.npy - Min: 1.3312461759020122, Max: 1.3847334887289884
Loaded and normalized AML3_cell16.npy - Min: 1.3311161422135425, Max: 1.3968166112149256
Loaded and normalized Tomogramma_BuddingYeastCell.npy - Min: 1.334, Max: 1.457687633726775
Loaded and normalized Tomogramma_Cell1.npy - Min: 1.334, Max: 1.455938645664837
Loaded and normalized Tomogramma_Cell2.npy - Min: 1.334, Max: 1.4541329102115372
Loaded and normalized Tomogramma_Cell3.npy - Min: 1.334, Max: 1.4414897865401213
Loaded and normalized tomo_Grafene_24h.npy - Min: 1.334, Max: 1.758458434932665
Loaded and normalized tomo_grafene_48h.npy - Min: -0.09675923117620404, Max: 0.30358731544428513
✅ Dictionary saved as MATLAB `.mat`: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\processed\all_normalizeddata.mat


In [34]:
import numpy as np
import os
import glob
import scipy.io as sio  # For saving .mat files
from sklearn.preprocessing import MinMaxScaler

# Set directory paths
data_folder = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1"
output_folder = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1"

# Ensure output directory exists
os.makedirs(output_folder, exist_ok=True)

# Function to load and normalize 3D NumPy data
def load_and_normalize_npy(file_path):
    """
    Loads a .npy file and applies Min-Max normalization.
    """
    data = np.load(file_path)  # Load the file
    min_val, max_val = np.min(data), np.max(data)  # Get min-max values
    normalized_data = (data - min_val) / (max_val - min_val)  # Normalize to [0,1]
    return normalized_data, min_val, max_val

# List all .npy files
npy_files = glob.glob(os.path.join(data_folder, "*.npy"))

# Dictionary to store the normalized datasets
normalized_data = {}
data_ranges = {}

# Process each .npy file
for file in npy_files:
    file_name = os.path.basename(file)
    base_name = os.path.splitext(file_name)[0]  # Remove .npy extension

    # Load and normalize data
    data, min_val, max_val = load_and_normalize_npy(file)
    normalized_data[file_name] = data  # Store in dictionary
    data_ranges[file_name] = (min_val, max_val)  # Store min/max values
    print(f"✅ Loaded and normalized {file_name} - Min: {min_val}, Max: {max_val}")

    # Save normalized data as .npy
    # npy_save_path = os.path.join(output_folder, f"{base_name}_normalized.npy")
    # np.save(npy_save_path, data)
    # print(f"📂 Saved: {npy_save_path}")

    # Save normalized data as .mat (MATLAB format)
    mat_save_path = os.path.join(output_folder, f"{base_name}_normalized.mat")
    sio.savemat(mat_save_path, {base_name: data})
    print(f"📂 Saved: {mat_save_path}")

print("\n🎉 All files processed and saved in 'data/processed' folder!")


✅ Loaded and normalized AML2_cell11.npy - Min: 1.3312461759020122, Max: 1.3847334887289884
📂 Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1\AML2_cell11_normalized.mat
✅ Loaded and normalized AML3_cell16.npy - Min: 1.3311161422135425, Max: 1.3968166112149256
📂 Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1\AML3_cell16_normalized.mat
✅ Loaded and normalized Tomogramma_BuddingYeastCell.npy - Min: 1.334, Max: 1.457687633726775
📂 Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1\Tomogramma_BuddingYeastCell_normalized.mat
✅ Loaded and normalized Tomogramma_Cell1.npy - Min: 1.334, Max: 1.455938645664837
📂 Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1\Tomogramma_Cell1_normalized.mat
✅ Loaded and normalized Tomogramma_Cell2.npy - Min

In [6]:
# import os
# import numpy as np
# import plotly.graph_objects as go
# import gc  # Garbage collector to free memory

# def plot3dinteractive(voldata, keyvalue, output_dir, sample_fraction=0.005):
#     """Plots large 3D NumPy arrays interactively and saves as HTML.
    
#     - `voldata`: Input 3D NumPy array.a
#     - `keyvalue`: Filename for saving.
#     - `output_dir`: Directory to save HTML plots.
#     - `sample_fraction`: Fraction of points to randomly plot.
#     """
#     array_3d = voldata
#     x1, y1, z1 = array_3d.shape
#     print(f"Shape of {keyvalue}: {x1, y1, z1}")

#     # Create a 3D meshgrid
#     x, y, z = np.meshgrid(np.arange(x1), np.arange(y1), np.arange(z1))

#     # Mask non-zero values
#     mask = array_3d > 0
#     x_vals = x[mask].flatten()
#     y_vals = y[mask].flatten()
#     z_vals = z[mask].flatten()
#     values = array_3d[mask].flatten()

#     # **Randomly sample points** to reduce memory usage
#     num_points = len(values)
#     sample_size = int(num_points * sample_fraction)

#     if sample_size > 0:
#         indices = np.random.choice(num_points, sample_size, replace=False)
#         x_vals = x_vals[indices]
#         y_vals = y_vals[indices]
#         z_vals = z_vals[indices]
#         values = values[indices]
#     else:
#         print(f"⚠ Warning: Not enough non-zero points for {keyvalue}. Skipping...")
#         return

#     print(f"Plotting {sample_size} points out of {num_points} ({sample_fraction * 100}% sampled)")

#     # Create a 3D scatter plot
#     fig = go.Figure(data=go.Scatter3d(
#         x=x_vals,
#         y=y_vals,
#         z=z_vals,
#         mode='markers',
#         marker=dict(
#             size=1,
#             color=values,
#             colorscale='Viridis',
#             opacity=0.5
#         )
#     ))

#     # Set axis labels
#     fig.update_layout(scene=dict(
#         xaxis_title='X',
#         yaxis_title='Y',
#         zaxis_title='Z'
#     ))

#     # **Save plot as an interactive HTML file**
#     save_path = os.path.join(output_dir, f"{keyvalue}.html")
#     fig.write_html(save_path)
#     print(f"✅ Saved: {save_path}")

#     # **Clear memory**
#     del fig, x_vals, y_vals, z_vals, values
#     gc.collect()  # Garbage collection to free memory

# # **Directory paths**
# data_dir = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1"
# output_dir = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw"

# # **Ensure output directory exists**
# os.makedirs(output_dir, exist_ok=True)

# # **Process all .npy files one by one**
# npy_files = [f for f in os.listdir(data_dir) if f.endswith(".npy")]

# for filename in npy_files:
#     file_path = os.path.join(data_dir, filename)
#     voldata = np.load(file_path)  # Load .npy file
#     plot3dinteractive(voldata, filename, output_dir, sample_fraction=0.04)  # Save & clear memory


In [7]:
import os
import numpy as np
import plotly.graph_objects as go
import gc  # Garbage collector to free memory

def plot3dinteractive(voldata, keyvalue, output_dir, sample_fraction=0.005):
    """Plots large 3D NumPy arrays interactively and saves as HTML.
    
    - `voldata`: Input 3D NumPy array.
    - `keyvalue`: Filename for saving.
    - `output_dir`: Directory to save HTML plots.
    - `sample_fraction`: Fraction of points to randomly plot.
    """
    array_3d = voldata
    x1, y1, z1 = array_3d.shape
    print(f"Shape of {keyvalue}: {x1, y1, z1}")

    # Create a 3D meshgrid
    x, y, z = np.meshgrid(np.arange(x1), np.arange(y1), np.arange(z1))

    # Mask non-zero values
    mask = array_3d > 0
    x_vals = x[mask].flatten()
    y_vals = y[mask].flatten()
    z_vals = z[mask].flatten()
    values = array_3d[mask].flatten()

    # **Randomly sample points** to reduce memory usage
    num_points = len(values)
    sample_size = int(num_points * sample_fraction)

    if sample_size > 0:
        indices = np.random.choice(num_points, sample_size, replace=False)
        x_vals = x_vals[indices]
        y_vals = y_vals[indices]
        z_vals = z_vals[indices]
        values = values[indices]
    else:
        print(f"⚠ Warning: Not enough non-zero points for {keyvalue}. Skipping...")
        return

    print(f"Plotting {sample_size} points out of {num_points} ({sample_fraction * 100}% sampled)")

    # **Enhanced Color Grading**
    colorscale = [
        [0.0, "white"],    # Outer structure (light color)
        [0.2, "lightblue"],
        [0.4, "deepskyblue"],
        [0.6, "dodgerblue"],
        [0.8, "blue"],      # Middle layers
        [1.0, "darkblue"]   # Deep inner structure (dark color)
    ]

    # Create a 3D scatter plot
    fig = go.Figure(data=go.Scatter3d(
        x=x_vals,
        y=y_vals,
        z=z_vals,
        mode='markers',
        marker=dict(
            size=2,
            color=values,
            colorscale=colorscale,
            opacity=0.5
        )
    ))

    # Set axis labels and layout
    fig.update_layout(
        title=f"3D Structure: {keyvalue}",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            bgcolor="black"  # Dark background for better contrast
        )
    )

    # **Save plot as an interactive HTML file**
    save_path = os.path.join(output_dir, f"{keyvalue}.html")
    fig.write_html(save_path)
    print(f"✅ Saved: {save_path}")

    # **Clear memory**
    del fig, x_vals, y_vals, z_vals, values
    gc.collect()  # Garbage collection to free memory

# **Directory paths**
data_dir = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\intermdata1"
output_dir = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw"

# **Ensure output directory exists**
os.makedirs(output_dir, exist_ok=True)

# **Process all .npy files one by one**
npy_files = [f for f in os.listdir(data_dir) if f.endswith(".npy")]

for filename in npy_files:
    file_path = os.path.join(data_dir, filename)
    voldata = np.load(file_path)  # Load .npy file
    plot3dinteractive(voldata, filename, output_dir, sample_fraction=0.05)  # Save & clear memory


Shape of AML2_cell11.npy: (200, 200, 200)
Plotting 400000 points out of 8000000 (5.0% sampled)
✅ Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw\AML2_cell11.npy.html
Shape of AML3_cell16.npy: (200, 200, 200)
Plotting 400000 points out of 8000000 (5.0% sampled)
✅ Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw\AML3_cell16.npy.html
Shape of Tomogramma_BuddingYeastCell.npy: (201, 201, 201)
Plotting 406030 points out of 8120601 (5.0% sampled)
✅ Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw\Tomogramma_BuddingYeastCell.npy.html
Shape of Tomogramma_Cell1.npy: (201, 201, 201)
Plotting 406030 points out of 8120601 (5.0% sampled)
✅ Saved: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\data\raw\Tomogramma_Cell1.npy.html
Shape of Tomogramma_Cell2.npy: (201, 201, 201)
Plotting 406030

In [16]:
import sys
import os
module_path = r"E:\Projects\substructure_3d_data\Substructure_Different_DataTypes\src\modules"
if not os.path.exists(module_path):
    print(f" i am looking for the Gaetano sys path")
    module_path = r"C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\src\modules"
    
# Add the module path to sys.path if it's nat already there
print(f"module path: {module_path}")
if module_path not in sys.path:
    sys.path.append(module_path)
    
from histogramplot import plot_normalizedata_hist
# from plot3dint import plot3dinteractive

for keyval in normalized_data:
    datakey = keyval
    print(f"\n data key :{datakey}")
    dataval = normalized_data[datakey]
    voldata=dataval
    keyvalue = datakey
    # plot3dinteractive(voldata,keyvalue)
    plot_normalizedata_hist(dataval,datakey)

 i am looking for the Gaetano sys path
module path: C:\Users\Gaetano\Desktop\create_with_codeRafi\MyProjects\Substructure_Different_DataTypes\src\modules

 data key :AML2_cell11.npy


IndexError: boolean index did not match indexed array along axis 0; size of axis is 201 but size of corresponding boolean axis is 200

In [5]:
# for k in normalized_data:
# #     print(f"{k} and val :  {normalized_data[k]}")
#     val = normalized_data[k]
#     val = val.flatten()
#     plt.figure()
#     sns.histplot(val,)
#     plt.title(k)
#     plt.ylim([0,0.2e6])
#     plt.show()

In [11]:
# x1,y1,z1 = (2,3,4)
# l = (21,23,44)
# print(f" direction :{x1,y1,z1}")
# for i in l:
#     print(i)
# # x1
# y1
# z1

 direction :(2, 3, 4)
21
23
44


In [14]:
z1

4